In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
import numpy as np
import pandas as pd

In [2]:
#Loading the dataset
boston_dataset= load_boston()
data= pd.DataFrame(data=boston_dataset.data,columns=boston_dataset.feature_names)
data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [3]:
features= data.drop(['INDUS','AGE'], axis=1)
log_prices=pd.DataFrame(np.log(boston_dataset.target),columns=['Prices'])

In [4]:
property_stats= np.ndarray(shape=(1,11))
property_stats=features.mean().values.reshape(1,11)
pd.DataFrame(property_stats,columns=[features.columns]).round(3)

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,3.614,11.364,0.069,0.555,6.285,3.795,9.549,408.237,18.456,356.674,12.653


In [5]:
regr=LinearRegression()
regration_results=regr.fit(features,log_prices)
table_fitted_resutls=pd.DataFrame(regration_results.coef_.reshape(11),columns=['coef'])

In [6]:
fitted_val=regration_results.predict(features)
Root_mean_squre=np.sqrt(mse(log_prices,fitted_val))


In [7]:
def Estimate_Price(numberOfRooms,StudentsPerClass,nextToRiver=False,highConfidance=True):
    """
    ** Estimating the price of a propraty in boston adjusted for inflation **
    Parameters
    ----------
    
    numberOfRooms:  int()
                    number of rooms per house (must be greater or equal to 1)
    
    StudentPerClass: int()
                     the teacher to student ratio per class/ number of student in a class (must be greater or equal to 1)
    
    nextToRiver*: bool() (Default False) 
                  proparty next to the charles river in boston 
    
    highCofidance*: bool() (Default True) 
                    estmation with 95% confidance 
    
    Returns
    -------
    String format with price estamtion:
                predicted price is X $ 
                in the range of -Y $ _ +Z $ 
    
    """
    if numberOfRooms<1 or StudentsPerClass<1:
        print('Impossi a khoya impossi')
        raise ValueError("params entered must be realistic 'be real'")
    #Cpi adjusting for inflation
    cpi=21.19
    #estimation price model 
    def get_log_estimate(nr_rooms,students_per_class,
                     next_to_river=False,
                     high_confidance=True,
                    ):
            
            #cofiguring params for our linear regression
            property_stats[0][4]=nr_rooms
            property_stats[0][8]=students_per_class
            if next_to_river:
                property_stats[0][2]=1
            else:
                property_stats[0][2]=0
            log_estimate=regr.predict(property_stats)
            if high_confidance:
                    Lower_bound=log_estimate-2*Root_mean_squre
                    Upper_bound=log_estimate+2*Root_mean_squre
            else:
                Lower_bound=log_estimate-Root_mean_squre
                Upper_bound=log_estimate+Root_mean_squre

            return [log_estimate[0][0],(Lower_bound[0][0],Upper_bound[0][0])]
    #formatin the answer to a readable string
    predicted_price,Range=get_log_estimate(numberOfRooms,StudentsPerClass,nextToRiver,highConfidance)
    lower_bound,upper_bound=Range
    print(f'predicted price is {round(np.e**predicted_price*10000*cpi)}$ \nin the range of -{round(np.e**lower_bound*10000*cpi)}$ _ +{round(np.e**upper_bound*10000*cpi)}$')
    

In [8]:
Estimate_Price(5,28,nextToRiver=False)

predicted price is 2723471$ 
in the range of -1871767$ _ +3962724$


In [9]:
mse(log_prices,regr.predict(features))

0.03516080084618688